In [1]:
%run init.ipynb

matchzoo version 1.0
`ranking_task` initialized with metrics [normalized_discounted_cumulative_gain@3(0.0), normalized_discounted_cumulative_gain@5(0.0), mean_average_precision(0.0)]
data loading ...
data loaded as `train_pack_raw` `dev_pack_raw` `test_pack_raw`


In [2]:
ranking_task = mz.tasks.Ranking(losses=mz.losses.RankCrossEntropyLoss(num_neg=1))
ranking_task.metrics = [
    mz.metrics.NormalizedDiscountedCumulativeGain(k=3),
    mz.metrics.NormalizedDiscountedCumulativeGain(k=5),
    mz.metrics.MeanAveragePrecision()
]

In [3]:
preprocessor = mz.models.MatchPyramid.get_default_preprocessor()

In [4]:
train_pack_processed = preprocessor.fit_transform(train_pack_raw)
dev_pack_processed = preprocessor.transform(dev_pack_raw)
test_pack_processed = preprocessor.transform(test_pack_raw)

Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 8090.96it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:03<00:00, 4945.43it/s]
Building Vocabulary from a datapack.: 100%|██████████| 418401/418401 [00:00<00:00, 2945067.59it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 7370.35it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:04<00:00, 4462.96it/s]
Processing length_right with len: 100%|██████████| 18841/18841 [00:00<00:00, 923867.82it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 122/122 [00:00<00:00, 9681.12it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 1115/1115 [00:00<00

In [5]:
preprocessor.context

{'filter_unit': <matchzoo.preprocessors.units.frequency_filter.FrequencyFilter at 0x7f9c35423810>,
 'vocab_unit': <matchzoo.preprocessors.units.vocabulary.Vocabulary at 0x7f9b56ce5510>,
 'vocab_size': 30058,
 'embedding_input_dim': 30058}

In [6]:
glove_embedding = mz.datasets.embeddings.load_glove_embedding(dimension=300)
term_index = preprocessor.context['vocab_unit'].state['term_index']
embedding_matrix = glove_embedding.build_matrix(term_index)
l2_norm = np.sqrt((embedding_matrix * embedding_matrix).sum(axis=1))
embedding_matrix = embedding_matrix / l2_norm[:, np.newaxis]

In [7]:
trainset = mz.dataloader.Dataset(
    data_pack=train_pack_processed,
    mode='pair',
    num_dup=2,
    num_neg=1
)
testset = mz.dataloader.Dataset(
    data_pack=test_pack_processed
)

In [8]:
padding_callback = mz.models.MatchPyramid.get_default_padding_callback()

trainloader = mz.dataloader.DataLoader(
    dataset=trainset,
    batch_size=20,
    stage='train',
    resample=True,
    sort=False,
    shuffle=True,
    callback=padding_callback
)
testloader = mz.dataloader.DataLoader(
    dataset=testset,
    batch_size=20,
    stage='dev',
    sort=False,
    shuffle=False,
    callback=padding_callback
)

In [9]:
model = mz.models.MatchPyramid()

model.params['task'] = ranking_task
model.params['embedding'] = embedding_matrix

model.params['kernel_count'] = [16, 32]
model.params['kernel_size'] = [[3, 3], [3, 3]]
model.params['dpool_size'] = [3, 10]
model.params['dropout_rate'] = 0.1

model.build()

print(model, sum(p.numel() for p in model.parameters() if p.requires_grad))

MatchPyramid(
  (embedding): Embedding(30058, 300)
  (matching): Matching()
  (conv2d): Sequential(
    (0): Sequential(
      (0): ConstantPad2d(padding=(0, 2, 0, 2), value=0)
      (1): Conv2d(1, 16, kernel_size=[3, 3], stride=(1, 1))
      (2): ReLU()
    )
    (1): Sequential(
      (0): ConstantPad2d(padding=(0, 2, 0, 2), value=0)
      (1): Conv2d(16, 32, kernel_size=[3, 3], stride=(1, 1))
      (2): ReLU()
    )
  )
  (dpool_layer): AdaptiveAvgPool2d(output_size=[3, 10])
  (dropout): Dropout(p=0.1, inplace=False)
  (out): Linear(in_features=960, out_features=1, bias=True)
) 9023161


In [10]:
optimizer = torch.optim.Adam(model.parameters())

trainer = mz.trainers.Trainer(
    model=model,
    optimizer=optimizer,
    trainloader=trainloader,
    validloader=testloader,
    validate_interval=None,
    epochs=5
)

In [11]:
trainer.run()

[Iter-102 Loss-0.632]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5599 - normalized_discounted_cumulative_gain@5(0.0): 0.6125 - mean_average_precision(0.0): 0.5815



[Iter-204 Loss-0.243]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.6082 - normalized_discounted_cumulative_gain@5(0.0): 0.6572 - mean_average_precision(0.0): 0.6115



[Iter-306 Loss-0.032]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5497 - normalized_discounted_cumulative_gain@5(0.0): 0.6012 - mean_average_precision(0.0): 0.5497



[Iter-408 Loss-0.011]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5517 - normalized_discounted_cumulative_gain@5(0.0): 0.6107 - mean_average_precision(0.0): 0.5628



[Iter-510 Loss-0.005]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5472 - normalized_discounted_cumulative_gain@5(0.0): 0.6119 - mean_average_precision(0.0): 0.5603

Cost time: 510.47750544548035s
